In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import numpy as np
import scprinter as scp
import scanpy as sc
import anndata
sc._settings.settings.n_jobs = -1

In [ ]:
work_dir = '/ewsc/rzhang/GAGE_seq_AD/scATAC'

In [ ]:
meta = pd.read_csv('snATAC_meta_TSS_6.tsv', sep='\t')
meta

In [ ]:
lib2whitelist = {}
for xx in meta['cellId']:
    xx = xx.split("#")
    lib = xx[0]
    bc = xx[1]
    if lib not in lib2whitelist:
        lib2whitelist[lib] = []
    lib2whitelist[lib].append(bc)

In [ ]:
libs = [...]
frags = [f'./fragments/{lib}_fragments.bed.gz' for lib in libs]
whitelists = [lib2whitelist[lib] for lib in libs]

In [ ]:
work_dir = '/ewsc/zhangruo/GAGE_seq_AD/scATAC'

In [ ]:
import scprinter as scp
import time
start = time.time()

if os.path.exists(f'{work_dir}/scprinter_tss6.h5ad'):
    printer = scp.load_printer(f'{work_dir}/scprinter_tss6.h5ad', scp.genome.hg38)
else:
    printer = scp.pp.import_fragments(
                        path_to_frags=frags,
                        barcodes=whitelists,
                        savename=f'{work_dir}/scprinter_tss6.h5ad',
                        genome=scp.genome.hg38,
                        min_num_fragments=0, min_tsse=0,
                        sorted_by_barcode=False, 
                        low_memory=False,
                        sample_names=libs
                        )
print ("takes", time.time()-start)

In [ ]:
from itertools import chain
all_whitelist = []
for lib, wl in zip(libs, whitelists):
    for x in wl:
        all_whitelist.append(f'{lib}_{x}')

In [ ]:
meta.index= [xx.split("#")[0] + "_" + xx.split("#")[1] for xx in meta['cellId']]
meta['index'] = meta.index
meta = meta.loc[all_whitelist].copy()
meta

In [ ]:
cell_grouping, group_names = scp.utils.df2cell_grouping(printer, meta[['index', 'Celltype']])

In [ ]:
scp.pp.call_peaks(
    printer=printer,
    frag_file=frags,
    cell_grouping=cell_grouping,
    group_names=list(group_names),
    iterative_peak_merging=True,
    merge_across_groups=True,
    preset='chromvar',
    sample_names=libs,
    overwrite=False,
    fdr_threshold=0.001)

In [ ]:
df = printer.uns['peak_calling']['merged']
peaks = pd.DataFrame(df, columns=df.columns)
peaks

In [ ]:
peaks.to_csv(f'{work_dir}/tss6_atac_peaks.bed', sep='\t', header=False, index=False)

In [ ]:
scp.pp.call_peaks(
    printer=printer,
    frag_file=frags,
    cell_grouping=cell_grouping,
    group_names=list(group_names),
    iterative_peak_merging=True,
    merge_across_groups=False,
    preset='chromvar',
    sample_names=libs,
    overwrite=False,
    fdr_threshold=0.001)

In [ ]:
cell_grouping, group_names = scp.utils.df2cell_grouping(printer, meta[meta['ADdiag3types'] == 'nonAD'][['index', 'Celltype']])
group_names = [f'CT_{x}' for x in group_names]

In [ ]:
scp.pp.call_peaks(
    printer=printer,
    frag_file=frags,
    cell_grouping=cell_grouping,
    group_names=list(group_names),
    iterative_peak_merging=True,
    merge_across_groups=False,
    preset='chromvar',
    sample_names=libs,
    overwrite=False,
    fdr_threshold=0.001)

In [ ]:
for key in printer.uns['peak_calling'].keys():
    if "cleaned" in key:
        df = printer.uns['peak_calling'][key]
        peaks = pd.DataFrame(df, columns=df.columns)
        # peaks.to_csv(f'{work_dir}/tss6_{key}_atac_peaks.bed', sep='\t', header=False, index=False)
        atac_adata = scp.pp.make_peak_matrix(printer=printer,
                                     regions=peaks,
                                     region_width=300,
                                     sparse=True)
        atac_adata.obs = meta.loc[atac_adata.obs.index].copy()
        atac_adata.write(f'{key}_cellxpeak.h5ad')